In [2]:
using EvolvingGraphs
using EvolvingGraphs.Centrality

In [1]:
workspace()

# Katz Motivation exmple 

In [7]:
g = EvolvingGraph{Node{String}, Int}()

Directed EvolvingGraph 0 nodes, 0 static edges, 0 timestamps

In [8]:
add_bunch_of_edges!(g, [("A", "B", 1), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 3)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [9]:
edges(g)

5-element Array{EvolvingGraphs.WeightedTimeEdge{EvolvingGraphs.Node{String},Int64,Float64},1}:
 Node(A)-1.0->Node(B) at time 1
 Node(A)-1.0->Node(C) at time 2
 Node(A)-1.0->Node(B) at time 2
 Node(C)-1.0->Node(A) at time 2
 Node(B)-1.0->Node(C) at time 3

In [10]:
katz(g)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.742301)
 (Node(B), 0.42943) 
 (Node(C), 0.514373)

In [11]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A", "B", 3), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [12]:
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.687679)
 (Node(B), 0.490062)
 (Node(C), 0.535666)

In [13]:
g3 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g3, [("A", "B", 100), ("A", "C", 2), ("A", "B", 2),("C", "A", 2),("B", "C", 1)])

Directed EvolvingGraph 3 nodes, 5 static edges, 3 timestamps

In [14]:
g1 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g1, [("A","B", 1),("B","C", 2)])
katz(g1)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.64654) 
 (Node(B), 0.604677)
 (Node(C), 0.465136)

In [15]:
g2 = EvolvingGraph{Node{String}, Int}()
add_bunch_of_edges!(g2, [("A","B", 2),("B","C", 1)])
katz(g2)

3-element Array{Tuple{EvolvingGraphs.Node{String},Float64},1}:
 (Node(A), 0.621164)
 (Node(B), 0.621164)
 (Node(C), 0.477818)

# Katz Centrality

Each node `v` has forword neighbours. Each forward neighbour has forword neighbour, and so on.

In [14]:
"""
distance between two active nodes

alpha: time unit parameter
"""
function temporal_distance(v1::TimeNode, v2::TimeNode, beta::Real = 1.)
    return node_key(v1) == node_key(v2) ? beta* abs(node_timestamp(v1) - node_timestamp(v2)) : 0
end
function temporal_distance(v1::Tuple{Int,Int}, v2::Tuple{Int,Int}, alpha::Real = 1.)
    return v1[1] == v2[1] ? alpha * abs(v1[2] - v2[2]) : 0
end

temporal_distance (generic function with 4 methods)

In [24]:
function temporal_katz(g::Union{AbstractEvolvingGraph, IntAdjacencyList}, v::Union{TimeNode, Tuple{Int,Int}}; alpha = 0.1, k = 5)
    score = 0.
    fronter = [v]
    level = 0
    println("start node $v")
    while level < k
        next = []
        println("at level $level, with $(length(fronter)) nodes")
        for u in fronter
            for v in forward_neighbors(g, u)
                push!(next, v)
                d = temporal_distance(v, u, 0.)
                d += level
                score += alpha^d
            end
        end
        fronter = next
        level += 1
    end
    return score
end

# Can be done in parallel, focus on useful node at certain time stamp


temporal_katz (generic function with 1 method)

In [17]:
active_nodes(g)

7-element Array{EvolvingGraphs.TimeNode{String,Int64},1}:
 TimeNode(A, 1)
 TimeNode(B, 1)
 TimeNode(A, 2)
 TimeNode(C, 2)
 TimeNode(B, 2)
 TimeNode(B, 3)
 TimeNode(C, 3)

In [19]:
an = active_nodes(g)

scores = Dict()
for n in an
    scores[n] = temporal_katz(g, n, alpha = 0.1, k = 100)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(A, 2) => 2.15152
  TimeNode(B, 3) => 1.0
  TimeNode(C, 3) => 0.0
  TimeNode(B, 1) => 0.23
  TimeNode(C, 2) => 1.31515
  TimeNode(B, 2) => 0.2
  TimeNode(A, 1) => 1.33815

In [29]:
an = active_nodes(g2)

scores = Dict()
for n in an
    scores[n] = katz_node(g2, n, alpha = 0.1, k = 100)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 3) => 0.0
  TimeNode(A, 2) => 2.33333
  TimeNode(C, 2) => 1.23333
  TimeNode(A, 3) => 1.0
  TimeNode(C, 1) => 0.223333
  TimeNode(B, 1) => 1.14233
  TimeNode(B, 2) => 0.1

In [27]:
an = active_nodes(g3)

scores = Dict()
for n in an
    scores[n] = katz_node(g3, n, alpha = 0.1, k = 100)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(A, 100) => 1.0
  TimeNode(B, 2)   => 1.0e-98
  TimeNode(B, 100) => 0.0
  TimeNode(B, 1)   => 1.12222
  TimeNode(C, 1)   => 0.222222
  TimeNode(A, 2)   => 2.22222
  TimeNode(C, 2)   => 1.22222

# Communicability Betweenness Centrality 

In [13]:
"""
Take account of all forward neighours.
"""
function temporal_katz_with_v_removed(g::EvolvingGraph, v; alpha = 0.1, k = 5)
    score = 0.
    fronter = [v]
    level = 0
    start = v
    while level < k
        next = []
        for u in fronter
            for v in forward_neighbors(g, u)
                if v != start
                    push!(next, v)
                    d = temporal_distance(v, u)
                    d += level
                    score += alpha^d
                end
            end
        end
        fronter = next
        level += 1
    end
    return score
end



function temporal_communicability(g::EvolvingGraph, v::TimeNode; alpha = 0.1, k = 5)
    
    score = temporal_katz(g, v, alpha = alpha, k = k)
    score_with_v_removed = temporal_katz_with_v_removed(g, v, alpha = alpha, k = k)
    
    r = (score - score_with_v_removed)/ score
    isnan(r)? 0 : r
end

temporal_communicability (generic function with 1 method)

In [14]:
an = active_nodes(g)

scores = Dict()
for n in an
    scores[n] = temporal_communicability(g, n, alpha = 0.1, k = 100)
end
scores

Dict{Any,Any} with 7 entries:
  TimeNode(B, 3) => 0.0
  TimeNode(A, 2) => 0.0564789
  TimeNode(C, 3) => 0
  TimeNode(A, 1) => 0.0
  TimeNode(B, 2) => 0.0
  TimeNode(B, 1) => 0.0
  TimeNode(C, 2) => 0.0860369

In [12]:
0/0

NaN

# Temporal Betweenness

In constrast with Katz and Communicability (which are based on walks), the following centrality algorithms are based on shortest paths.

# Temporal Closeness Centrality 

# Temporal PageRank

Random walk interpretation can not travel back in time.

* Block matrix version (use reverse pagerank): useful for evolving graph with a small number of timestamps

* Probability based (time-stamps based rating): Random walk interpretation. Not fixed probability for all nodes but different probability for each individual node. 

# JMLR and Random Evolving Graph Data

Start with the original definition and compare the change of ranking of top authors if we vary

In [6]:
g = EvolvingGraph()
g = random_evolving_graph(g, 100, 5, 0.2)

Directed EvolvingGraph 100 nodes, 9810 static edges, 5 timestamps

In [7]:
r = katz(g)
sorted_top = sort(r, by = x -> x[2], rev = true)[1:20]

20-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(6), 0.229854)  
 (Node(54), 0.220491) 
 (Node(43), 0.193352) 
 (Node(55), 0.187044) 
 (Node(73), 0.1859)   
 (Node(80), 0.185528) 
 (Node(33), 0.168963) 
 (Node(37), 0.146367) 
 (Node(94), 0.143335) 
 (Node(17), 0.134177) 
 (Node(45), 0.125457) 
 (Node(52), 0.125012) 
 (Node(11), 0.11887)  
 (Node(36), 0.114556) 
 (Node(25), 0.10687)  
 (Node(46), 0.10678)  
 (Node(89), 0.096828) 
 (Node(23), 0.0874197)
 (Node(34), 0.077072) 
 (Node(60), 0.0741519)

In [8]:
sorted_top_keys = map(x -> node_key(x[1]), sorted_top)

20-element Array{Int64,1}:
  6
 54
 43
 55
 73
 80
 33
 37
 94
 17
 45
 52
 11
 36
 25
 46
 89
 23
 34
 60

In [9]:
intg = evolving_graph_to_adj(g)

Directed IntAdjacencyList (100 nodes, 9810 static edges, 5 timestamps)

In [25]:
scores = []
for n in active_nodes(g)
    if node_key(n) in sorted_top_keys
        intn = (node_key(n),node_timestamp(n))
        println(intn)
        tr = temporal_katz(intg, intn, alpha = 0.1, k = 5)
        push!(scores, (n, tr))
    end
end
scores

(11, 1)
start node (11, 1)
at level 0, with 1 nodes
at level 1, with 27 nodes
at level 2, with 599 nodes
at level 3, with 13701 nodes
at level 4, with 309374 nodes
(17, 1)
start node (17, 1)
at level 0, with 1 nodes
at level 1, with 25 nodes
at level 2, with 570 nodes
at level 3, with 13042 nodes
at level 4, with 294318 nodes
(45, 1)
start node (45, 1)
at level 0, with 1 nodes
at level 1, with 28 nodes
at level 2, with 633 nodes
at level 3, with 14486 nodes
at level 4, with 326953 nodes
(52, 1)
start node (52, 1)
at level 0, with 1 nodes
at level 1, with 20 nodes
at level 2, with 449 nodes
at level 3, with 10210 nodes
at level 4, with 229036 nodes
(25, 1)
start node (25, 1)
at level 0, with 1 nodes
at level 1, with 17 nodes
at level 2, with 386 nodes
at level 3, with 8759 nodes
at level 4, with 196848 nodes
(34, 1)
start node (34, 1)
at level 0, with 1 nodes
at level 1, with 26 nodes
at level 2, with 605 nodes
at level 3, with 13884 nodes
at level 4, with 313099 nodes
(36, 1)
start nod

at level 4, with 257955 nodes
(60, 3)
start node (60, 3)
at level 0, with 1 nodes
at level 1, with 23 nodes
at level 2, with 502 nodes
at level 3, with 11021 nodes
at level 4, with 238014 nodes
(6, 3)
start node (6, 3)
at level 0, with 1 nodes
at level 1, with 19 nodes
at level 2, with 415 nodes
at level 3, with 9028 nodes
at level 4, with 195055 nodes
(45, 3)
start node (45, 3)
at level 0, with 1 nodes
at level 1, with 25 nodes
at level 2, with 572 nodes
at level 3, with 12400 nodes
at level 4, with 267808 nodes
(89, 3)
start node (89, 3)
at level 0, with 1 nodes
at level 1, with 27 nodes
at level 2, with 576 nodes
at level 3, with 12478 nodes
at level 4, with 270099 nodes
(36, 3)
start node (36, 3)
at level 0, with 1 nodes
at level 1, with 25 nodes
at level 2, with 544 nodes
at level 3, with 11824 nodes
at level 4, with 255260 nodes
(43, 3)
start node (43, 3)
at level 0, with 1 nodes
at level 1, with 25 nodes
at level 2, with 567 nodes
at level 3, with 12400 nodes
at level 4, with 26

100-element Array{Any,1}:
 (TimeNode(11, 1), 1224.83)
 (TimeNode(17, 1), 1163.85)
 (TimeNode(45, 1), 1293.58)
 (TimeNode(52, 1), 906.064)
 (TimeNode(25, 1), 778.205)
 (TimeNode(34, 1), 1238.17)
 (TimeNode(36, 1), 1539.86)
 (TimeNode(54, 1), 1144.23)
 (TimeNode(55, 1), 1169.12)
 (TimeNode(43, 1), 1099.13)
 (TimeNode(80, 1), 1363.9) 
 (TimeNode(23, 1), 1047.99)
 (TimeNode(6, 1), 1296.96) 
 ⋮                         
 (TimeNode(36, 5), 535.924)
 (TimeNode(45, 5), 521.451)
 (TimeNode(55, 5), 701.138)
 (TimeNode(80, 5), 709.019)
 (TimeNode(43, 5), 504.202)
 (TimeNode(37, 5), 374.039)
 (TimeNode(94, 5), 604.201)
 (TimeNode(89, 5), 317.046)
 (TimeNode(60, 5), 461.538)
 (TimeNode(73, 5), 602.783)
 (TimeNode(25, 5), 424.007)
 (TimeNode(17, 5), 592.935)

In [26]:
using DataStructures

sum_scores = DefaultDict(0.) 
for (n, v) in scores
    sum_scores[node_key(n)] += v
end
sum_scores 

DataStructures.DefaultDict{Any,Any,Float64} with 20 entries:
  54 => 3890.16
  80 => 5065.66
  89 => 4106.01
  11 => 4020.13
  46 => 4404.96
  43 => 4624.15
  25 => 3715.35
  55 => 4597.42
  34 => 4169.42
  60 => 4085.78
  17 => 4135.5
  6  => 3774.14
  73 => 4124.62
  37 => 3844.12
  45 => 4557.3
  23 => 3775.93
  36 => 4394.77
  94 => 4141.0
  52 => 3909.63
  33 => 3519.81

In [31]:
accu_scores = DefaultDict([0.]) 
for (n, v) in scores
#     println("node $n")
#     println("value $v")
    push!(accu_scores[node_key(n)], v)
#     push!(accu_scores[node_key(n)], accu_scores[node_key(n)][end] + v)
end
accu_scores

node TimeNode(11, 1)
value 1224.834899819936
node TimeNode(17, 1)
value 1163.8518998292761
node TimeNode(45, 1)
value 1293.5838998093886
node TimeNode(52, 1)
value 906.0639998690834
node TimeNode(25, 1)
value 778.2052998885803
node TimeNode(34, 1)
value 1238.168899817753
node TimeNode(36, 1)
value 1539.8606997719116
node TimeNode(54, 1)
value 1144.227099832334
node TimeNode(55, 1)
value 1169.1183998284694
node TimeNode(43, 1)
value 1099.1326998390223
node TimeNode(80, 1)
value 1363.896299798577
node TimeNode(23, 1)
value 1047.9910998470273
node TimeNode(6, 1)
value 1296.9582998088517
node TimeNode(37, 1)
value 1228.5242998193146
node TimeNode(46, 1)
value 910.8328998682736
node TimeNode(60, 1)
value 1141.6199998327547
node TimeNode(89, 1)
value 1341.6504998016464
node TimeNode(33, 1)
value 922.5920998664808
node TimeNode(73, 1)
value 900.4506998700047
node TimeNode(94, 1)
value 1128.5436998347964
node TimeNode(11, 2)
value 746.1284998952133
node TimeNode(17, 2)
value 910.4348998706787


DataStructures.DefaultDict{Any,Any,Array{Float64,1}} with 0 entries

In [30]:
using Plots

INFO: Precompiling module Plots.


In [27]:
r2 = collect(sum_scores);

In [28]:
sorted_r2 = sort(r2, by = x -> x[2], rev = true)

20-element Array{Pair{Any,Any},1}:
 Pair{Any,Any}(80, 5065.66)
 Pair{Any,Any}(43, 4624.15)
 Pair{Any,Any}(55, 4597.42)
 Pair{Any,Any}(45, 4557.3) 
 Pair{Any,Any}(46, 4404.96)
 Pair{Any,Any}(36, 4394.77)
 Pair{Any,Any}(34, 4169.42)
 Pair{Any,Any}(94, 4141.0) 
 Pair{Any,Any}(17, 4135.5) 
 Pair{Any,Any}(73, 4124.62)
 Pair{Any,Any}(89, 4106.01)
 Pair{Any,Any}(60, 4085.78)
 Pair{Any,Any}(11, 4020.13)
 Pair{Any,Any}(52, 3909.63)
 Pair{Any,Any}(54, 3890.16)
 Pair{Any,Any}(37, 3844.12)
 Pair{Any,Any}(23, 3775.93)
 Pair{Any,Any}(6, 3774.14) 
 Pair{Any,Any}(25, 3715.35)
 Pair{Any,Any}(33, 3519.81)

In [17]:
sorted_top

20-element Array{Tuple{EvolvingGraphs.Node{Int64},Float64},1}:
 (Node(6), 0.229854)  
 (Node(54), 0.220491) 
 (Node(43), 0.193352) 
 (Node(55), 0.187044) 
 (Node(73), 0.1859)   
 (Node(80), 0.185528) 
 (Node(33), 0.168963) 
 (Node(37), 0.146367) 
 (Node(94), 0.143335) 
 (Node(17), 0.134177) 
 (Node(45), 0.125457) 
 (Node(52), 0.125012) 
 (Node(11), 0.11887)  
 (Node(36), 0.114556) 
 (Node(25), 0.10687)  
 (Node(46), 0.10678)  
 (Node(89), 0.096828) 
 (Node(23), 0.0874197)
 (Node(34), 0.077072) 
 (Node(60), 0.0741519)

In [20]:
using EvolvingGraphs
using EvolvingGraphs.Centrality
import EzXML


# load graph data at each year
# form evolving graph
g = EvolvingGraph{Node{String}, Int}()

for year in range(2001, 17)
    sg = load_graphml("jmlr_$(year).graphml")
    add_graph!(g, sg, year)
end
g

Directed EvolvingGraph 3354 nodes, 15654 static edges, 17 timestamps

In [6]:
rating = katz(g, 0.1, 0.01; mode = :receive)


sorted_authors = sort(rating, by = x -> x[2], rev = true)

println("Top 10 rating authors")
println(sorted_authors[1:10])

println("Button 10 rating authors")
println(sorted_authors[end-10:end])

Top 10 rating authors
Tuple{EvolvingGraphs.Node{String},Float64}[(Node(Manuel Gomez-Rodriguez), 1.0), (Node(Henry Adams), 0.984369), (Node(Joseph Bradley), 0.972919), (Node(Le Song), 0.959548), (Node(Wei-Sheng Chin), 0.937996), (Node(Zoubin Ghahramani), 0.891851), (Node(Lori Ziegelmeier), 0.885932), (Node(Francis Motta), 0.885932), (Node(Eric Hanson), 0.885932), (Node(Tegan Emerson), 0.885932)]
Button 10 rating authors
Tuple{EvolvingGraphs.Node{String},Float64}[(Node(Sa, Virginia R. de), 2.49324e-5), (Node(Chapados, Nicolas), 2.49324e-5), (Node(Forman, George), 2.49324e-5), (Node(Pla, Ferran), 2.49299e-5), (Node(Gavinsky, Dmitry), 2.49299e-5), (Node(Zhang, Huajie), 2.49299e-5), (Node(Wrobel, Stefan), 2.49299e-5), (Node(Vempala, Santosh), 2.49299e-5), (Node(Veloso, Manuela), 2.49299e-5), (Node(Struyf, Jan), 2.49299e-5), (Node(Barto, Andrew G.), 2.49299e-5)]


In [37]:
an = active_nodes(g)[1:11]

scores = Dict()
for n in an
    scores[n] = temporal_katz(g, n, alpha = 0.1, k = 5)
end
scores

start node TimeNode(Manevitz, Larry M., 2001)
at level 0, with 1 nodes
at level 1, with 2 nodes
at level 2, with 3 nodes
at level 3, with 5 nodes
at level 4, with 8 nodes
start node TimeNode(Yousef, Malik, 2001)
at level 0, with 1 nodes
at level 1, with 1 nodes
at level 2, with 2 nodes
at level 3, with 3 nodes
at level 4, with 5 nodes
start node TimeNode(Gates, Kevin E., 2001)
at level 0, with 1 nodes
at level 1, with 2 nodes
at level 2, with 6 nodes
at level 3, with 19 nodes
at level 4, with 62 nodes
start node TimeNode(Masters, Annette, 2001)
at level 0, with 1 nodes
at level 1, with 2 nodes
at level 2, with 6 nodes
at level 3, with 19 nodes
at level 4, with 62 nodes
start node TimeNode(Downs, Tom, 2001)
at level 0, with 1 nodes
at level 1, with 4 nodes
at level 2, with 13 nodes
at level 3, with 43 nodes
at level 4, with 145 nodes
start node TimeNode(Vapnik, Vladimir, 2001)
at level 0, with 1 nodes
at level 1, with 3 nodes
at level 2, with 11 nodes
at level 3, with 40 nodes
at level 

Dict{Any,Any} with 11 entries:
  TimeNode(Manevitz, Larry M., 2001)  => 2.3593
  TimeNode(Singer, Yoram, 2001)       => 4.86033
  TimeNode(Downs, Tom, 2001)          => 5.9246
  TimeNode(Vapnik, Vladimir, 2001)    => 4.6955
  TimeNode(Crammer, Koby, 2001)       => 6.33534
  TimeNode(Horn, David, 2001)         => 4.6955
  TimeNode(Masters, Annette, 2001)    => 2.8727
  TimeNode(Ben-Hur, Asa, 2001)        => 7.7474
  TimeNode(Yousef, Malik, 2001)       => 1.2358
  TimeNode(Gates, Kevin E., 2001)     => 2.8727
  TimeNode(Siegelmann, Hava T., 2001) => 4.6955